In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "bears"})

"Why don't bears ever wear shoes?\n\nBecause they have bear feet!"

In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_texts(
    ['harrison worked at kensho', 'bears like to eat honey'],
    embedding = OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [6]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [7]:
retriever.get_relevant_documents("What do bears like to eat?")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [9]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']),
    "question": lambda x: x['question']
}) | prompt | model | output_parser

In [11]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [16]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [19]:
runnable = prompt | model

runnable.invoke({"input": "What is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'weather_search'}})

In [20]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [21]:
model = model.bind(functions=functions)

runnable = prompt | model

runnable.invoke({"input": "How did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "team_name": "patriots"\n}', 'name': 'sports_search'}})

In [23]:
from langchain.llms import OpenAI
import json

simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

In [24]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [25]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [26]:
final_chain = simple_chain.with_fallbacks([chain])

final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [28]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "bears"})

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

In [29]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears ever wear shoes?\n\nBecause they have bear feet!",
 "Why don't frogs make good lawyers?\n\nBecause they're always too jumpy!"]

In [30]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



In [31]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes?\n\nBecause they have bear feet!"